In [1]:
import autosklearn.classification
import pandas as pd
from joblib import dump, load
import numpy as np
import glob

/opt/conda/lib/python3.7/site-packages/pyparsing.py:3174: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [2]:
models = []
for model_path in glob.glob("model_3cv/model_run_*"):
    models.append(load(model_path))

In [3]:
bad_models = []
for model_path in glob.glob("model_3cv/bad_auto_model_run_*"):
    bad_models.append(load(model_path))

In [4]:
numeric_conv_cols = ['age', 'age_o', 'importance_same_race', 'importance_same_religion', 'pref_o_attractive', 'pref_o_sincere',\
                    'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_o', 'sinsere_o', \
                    'intelligence_o', 'funny_o', 'ambitous_o', 'shared_interests_o', 'attractive_important', 'sincere_important', \
                    'intellicence_important', 'funny_important', 'ambtition_important', 'shared_interests_important', \
                    'attractive', 'sincere', 'intelligence', 'funny', 'ambition', 'attractive_partner', 'sincere_partner', \
                     'intelligence_partner', 'funny_partner', 'ambition_partner', 'shared_interests_partner', 'sports', \
                     'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', \
                     'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'interests_correlate', 'expected_happy_with_sd_people', \
                    'expected_num_interested_in_me', 'expected_num_matches', 'like', 'guess_prob_liked', 'met', 'gender']

In [5]:
df = pd.read_csv("speeddating_reduced.csv")
X= df[numeric_conv_cols].fillna(-1) # required for DT
y= df['match']

#### Check for which instances the models disagree

In [6]:
y_pred = np.stack([model.predict(X) for model in models + bad_models])
df_pred = pd.DataFrame(y_pred).T
df_pred.columns = ['good_1', 'good_2', 'good_3', 'bad_1', 'bad_2', 'bad_3']
df_pred.head()

,good_1,good_2,good_3,bad_1,bad_2,bad_3
0,0,1,1,0,0,0
1,1,0,0,0,0,0
2,1,1,1,0,1,0
3,1,1,1,1,1,0
4,1,1,0,1,0,0


In [7]:
df_pred[(df_pred.good_1 + df_pred.good_2 + df_pred.good_3 ==3) & (y==1) & (df_pred.bad_1 + df_pred.bad_2 + df_pred.bad_3 == 0)] #-> use 8362

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,good_1,good_2,good_3,bad_1,bad_2,bad_3
58,1,1,1,0,0,0
71,1,1,1,0,0,0
81,1,1,1,0,0,0
82,1,1,1,0,0,0
83,1,1,1,0,0,0
...,...,...,...,...,...,...
8049,1,1,1,0,0,0
8215,1,1,1,0,0,0
8303,1,1,1,0,0,0
8324,1,1,1,0,0,0


In [8]:
y_pred_proba = [model.predict_proba(X[8362:8363])[0][1] for model in models + bad_models]
' & '.join([str(np.round(y,2)) for y in y_pred_proba])

'0.58 & 0.54 & 0.57 & 0.49 & 0.29 & 0.34'